In [7]:
from hdrh.histogram import HdrHistogram
import seaborn as sns
import pandas
import pandas as pd
from matplotlib import pyplot as plt
import os.path
from enum import Enum
import matplotlib as mpl
from typing import *

INVOCATIONS=3

def load_data(invocation: int, folder: str):
    path = os.path.realpath(os.path.expanduser('{}.{}/pauses.csv'.format(folder, invocation)))
    if not os.path.isfile(path):
        return None
    df =  pandas.read_csv(path, names=["nanos"])
    df["millis"] = df["nanos"] / 1000000
    df["micros"] = df["nanos"] / 1000
    return df


def load_and_dump_data(name: str, runid: str, buildstring: str):
    folder = f'~/MMTk-Dev/evaluation/results/log/{runid}/{buildstring}'
    dfs = []
    for i in range(INVOCATIONS):
        dfs.append(load_data(i, folder))
    df = pd.concat(dfs, ignore_index=True)
    percentiles = {
        '50': df['millis'].quantile(0.5),
        '99': df['millis'].quantile(0.99),
        '99.9': df['millis'].quantile(0.999),
        '99.99': df['millis'].quantile(0.9999),
    }
    print(name + ':    ' + '  '.join([f'{k}%={x:.1f}' for k, x in percentiles.items()]))
    # print(name + ':    ' + ' & '.join([f'{x:.1f}' for _, x in percentiles.items()]))

In [8]:
RUNID = 'elk-2022-03-21-Mon-062551'
RUNID_10X = 'elk-2022-03-21-Mon-042713'

load_and_dump_data(name='G1', runid=RUNID, buildstring='lusearch.2000.106.jdk-lxr.g1.common.hs_pauses.dacapochopin-29a657f')
load_and_dump_data(name='Shen.', runid=RUNID, buildstring='lusearch.2000.106.jdk-lxr.shenandoah.common.hs_pauses.dacapochopin-29a657f')
load_and_dump_data(name='Shen. 7/8', runid=RUNID, buildstring='lusearch.2000.106.jdk-lxr.shenandoah.common.hs_pauses.hs_cgc-21.dacapochopin-29a657f')
load_and_dump_data(name='Shen. 10x', runid=RUNID_10X, buildstring='lusearch.10000.530.jdk-lxr.shenandoah.common.hs_pauses.dacapochopin-29a657f')
load_and_dump_data(name='LXR', runid=RUNID, buildstring='lusearch.2000.106.jdk-lxr.ix.common.tph.nr-1.dacapochopin-29a657f')

G1:    50%=0.6  99%=1.0  99.9%=1.2  99.99%=1.6
Shen.:    50%=0.1  99%=0.5  99.9%=0.6  99.99%=2.2
Shen. 7/8:    50%=0.2  99%=0.4  99.9%=0.6  99.99%=1.4
Shen. 10x:    50%=0.2  99%=0.6  99.9%=0.7  99.99%=0.8
LXR:    50%=2.2  99%=3.5  99.9%=8.1  99.99%=11.9
